In [1]:
%matplotlib inline
import numpy as np                                    #arrays and matrix math
import pandas as pd                                   #work with DataFrames
import matplotlib.pyplot as plt                       #plotting and visualization
import matplotlib.dates as mdates                     #datetime formate in plots

import h5py                                           #import h5 files
import os                                             #OS operations
import scipy.signal as signal                         #signal processing
from scipy.io import loadmat                          #load MatLab m-files

from skimage.metrics import mean_squared_error as image_mse         #Mean Squared Error
from skimage.metrics import structural_similarity as image_ssim     #Structural Similarity Index
from skimage.metrics import peak_signal_noise_ratio as image_psnr   #Peak Signal-Noise Ratio

# Define arguments for text box in PLT.TEXT()
my_box = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Check tensorflow GPU settings
import tensorflow as tf
sys_info = tf.sysconfig.get_build_info()
print('Tensorflow built with CUDA?',  tf.test.is_built_with_cuda())
print('Devices available:', tf.config.experimental.list_physical_devices())
print('# GPU available:', len(tf.config.experimental.list_physical_devices('GPU')))
print("CUDA: {} | cuDNN: {}".format(sys_info["cuda_version"], sys_info["cudnn_version"]))
tf.config.list_physical_devices()

Tensorflow built with CUDA? True
Devices available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]
# GPU available: 1
CUDA: 64_101 | cuDNN: 64_7


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

***

In [2]:
# EXPERIMENT 54 ONLY
data_exp54 = pd.read_pickle('data_exp54.pkl')
print('Experiment 54 shape: {}'.format(data_exp54.shape))

time_complete_exp54 = np.array([loadmat('time_complete_54.mat')['time']], dtype='datetime64[us]').reshape(-1)
print('Times for Experiment 54 shape: {}'.format(time_complete_exp54.shape))

trange_54 = mdates.date2num(time_complete_exp54)
xrange_54 = np.arange(4950, 5150)
print('Timerange 54: {}'.format(trange_54.shape))
print('X-range 54: {}'.format(xrange_54.shape))

Experiment 54 shape: (200, 4200000)
Times for Experiment 54 shape: (4200000,)
Timerange 54: (4200000,)
X-range 54: (200,)


In [ ]:
# fig, ax1 = plt.subplots(1, 1, figsize=(20,10))
# date_format = mdates.DateFormatter('%H:%M:%S')

# im1 = ax1.imshow(data_exp54, aspect='auto', cmap='seismic', vmin=-50, vmax=50, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
# ax1.xaxis_date(); ax1.xaxis.set_major_formatter(date_format)
# ax1.set_xlabel('Time [s] on 06/20/2019'); ax1.set_ylabel('Distance [m]'); ax1.set_title('Experiment 54')
# plt.colorbar(im1, ax=ax1)

# fig.autofmt_xdate()
# plt.show();

***

FFT (Fast Fourier Transform)

In [ ]:
from scipy.fft import fft2 as FFT2 
from scipy.fft import ifft2 as iFFT2 
from scipy.fft import fftshift as Shift
from scipy.fft import ifftshift as iShift

In [ ]:
def compression_shapes(dat, cr):
    # percentage to be used from the compression ratio
    perc = np.sqrt(cr)
    
    # distance boundaries
    x1 = int((dat.shape[0]/2) - (dat.shape[0]*perc/2))
    x2 = int((dat.shape[0]/2) + (dat.shape[0]*perc/2))
    
    # time boundaries
    t1 = int((dat.shape[1]/2) - (dat.shape[1]*perc/2))
    t2 = int((dat.shape[1]/2) + (dat.shape[1]*perc/2))
   
    # return
    return x1, x2, t1, t2, perc, cr

In [ ]:
def do_FFT(data, compression_ratio):
    data_exp = np.array(data) #make data np.array
    data_fft = FFT2(data_exp) #take the FFT
    data_fft_shift = Shift(data_fft) #center shift the FFT
    
    x1, x2, t1, t2, perc, cr = compression_shapes(data_fft, compression_ratio) #compression dimensions
    ranges = [x1, x2, t1, t2]
    
    data_fft_shift_center = np.zeros((data_exp.shape[0], data_exp.shape[-1]), dtype='csingle') #make empty matrix
    data_fft_shift_center[x1:x2, t1:t2] = data_fft_shift[x1:x2, t1:t2] #fill empty matrix at compression dimensions
    
    data_fft_ishift = iShift(data_fft_shift_center) #inverse shift
    data_ifft = iFFT2(data_fft_ishift).real #inverse FFT
    
    data_exp_norm   = data_exp  / np.max(np.abs(data_exp))  #normalize original data
    data_ifft_norm  = data_ifft / np.max(np.abs(data_ifft)) #normalize reconstructed data

    mse  = image_mse(data_exp_norm,  data_ifft_norm) #mse
    ssim = image_ssim(data_exp_norm, data_ifft_norm) #ssim
    psnr = image_psnr(data_exp_norm, data_ifft_norm) #psnr
    metrics = [mse, ssim, psnr]
    
    return data_ifft, data_fft_shift, ranges, metrics

In [ ]:
def make_FFT_plots():
    fig, ax1 = plt.subplots(1, 1, figsize=(8,8))
    date_format = mdates.DateFormatter('%H:%M:%S')
    im1 = ax1.imshow(np.log10(np.abs(data_fft_shift)), aspect='auto', cmap='seismic')
    ax1.set_xlabel('$Z_t$');        ax1.set_ylabel('$Z_d$')
    ax1.set_xticks([]);             ax1.set_yticks([])
    x1, x2, t1, t2 = ranges
    ax1.hlines(x1, t1, t2, 'k');    ax1.hlines(x2, t1, t2, 'k')
    ax1.vlines(t1, x1, x2, 'k');    ax1.vlines(t2, x1, x2, 'k')
    fig.autofmt_xdate()
    plt.show();
    
    fig, ax1  = plt.subplots(1, 1, figsize=(12,6))
    date_format = mdates.DateFormatter('%H:%M:%S')
    im1 = ax1.imshow(data_ifft, aspect='auto', cmap='seismic', vmin=-50, vmax=50, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
    ax1.xaxis_date(); ax1.xaxis.set_major_formatter(date_format)
    ax1.set_xlabel('Time [s] on 06/20/2019'); ax1.set_ylabel('Distance [m]') 
    ax1.set_title('FFT-Reconstructed Experiment 54')
    fig.autofmt_xdate()
    plt.show();

In [ ]:
data_ifft, data_fft_shift, ranges, metrics = do_FFT(data_exp54, 0.005)

In [ ]:
print('MSE: {:.3e}'.format(metrics[0]))
print('SSIM: {:.2f}'.format(metrics[1]*100))
print('PSNR: {:.2f}'.format(metrics[2]))

In [ ]:
make_FFT_plots()

***

# END